In [ ]:
#default_exp exhaustive_training

# 70-exhaustive-training
> Create systematic testing procedure and document conclusions

**Purpose**  The purpose of this notebook is to provide comprehensive testing with respect to:
1. Hyper-parameters (learning rate, dropout rate, compounding, etc)
2. Training data: this includes varying language (Spanish, Portuguese) and record type (baptismal, burial, and marriage)

Errors I'm Getting:
1. C:\Users\14193\anaconda3\lib\site-packages\spacy\language.py:639: UserWarning: [W033] Training a new parser or NER using a model with no lexeme normalization table. This may degrade the performance of the model to some degree. If this is intentional or the language you're using doesn't have a normalization table, please ignore this warning. If this is surprising, make sure you have the spacy-lookups-data package installed. The languages with lexeme normalization tables are currently: da, de, el, en, id, lb, pt, ru, sr, ta, th.
  **kwargs
2. C:\Users\14193\anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "[margin]: Fran.co esc.o Domingo dos de Marco de mi..." with entities "[(10, 17, 'PER'), (18, 23, 'CHAR'), (24, 78, 'DATE...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)

In [ ]:
#no_test

#data structure imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy

#python imports
import random

#modeling imports
from spacy.util import fix_random_seed
from ssda_nlp.collate import *
from ssda_nlp.split_data import *
from ssda_nlp.modeling import *
from ssda_nlp.preprocessing import *
from ssda_nlp.model_performance_utils import *

In [ ]:
#no_test
#if you want the same model everytime:
seed = 2436
random.seed(seed)
fix_random_seed(seed)

# Available Training Data
    1. 15834_annot.jsonl:                Baptism, Cuba
    2. 239746_annot.jsonl:               Baptism, Florida
    3. 419324_annot.jsonl:               Baptism, Colombia
    4. 701075_annot.jsonl:               Baptism, Brazil
    5. deathandbirthdata.jsonl:          ?
    6. guatemala.annotations.jsonl:      ?

In [ ]:
#no_test

collated_df1 = prodigy_output_to_collated_df("transcriptions/15834_annot.jsonl")
collated_df2 = prodigy_output_to_collated_df("transcriptions/239746_annot.jsonl")
collated_df3 = prodigy_output_to_collated_df("transcriptions/419324_annot.jsonl")

collated_df = collated_df1

print(collated_df1.shape)
print(collated_df2.shape)
print(collated_df3.shape)

display(collated_df1.head())
display(collated_df2.head())
display(collated_df3.head())

(2206, 6)
(2839, 6)
(947, 6)


,entry_no,text,entity,start,end,label
0,1,[margin]: Juana. Esc.va Domingo veinte y dos d...,Juana,10,15,PER
1,1,[margin]: Juana. Esc.va Domingo veinte y dos d...,Esc.va,17,23,CHAR
2,1,[margin]: Juana. Esc.va Domingo veinte y dos d...,Domingo veinte y dos de [roto] y nueve,24,62,DATE
3,1,[margin]: Juana. Esc.va Domingo veinte y dos d...,Thomas de Orvera,66,82,PER
4,1,[margin]: Juana. Esc.va Domingo veinte y dos d...,Juana,121,126,PER


,entry_no,text,entity,start,end,label
0,1,"1. María Dolores Sanchez Lunes, día veinte y u...",María Dolores Sanchez,3,24,PER
1,1,"1. María Dolores Sanchez Lunes, día veinte y u...","Lunes, día veinte y uno de Octubre de Mil Sete...",25,94,DATE
2,1,"1. María Dolores Sanchez Lunes, día veinte y u...",Don Miguel o’Reilly,98,117,PER
3,1,"1. María Dolores Sanchez Lunes, día veinte y u...","Teniente de Cura Beneficiado,",118,147,CHAR
4,1,"1. María Dolores Sanchez Lunes, día veinte y u...",Vicario,148,155,CHAR


,entry_no,text,entity,start,end,label
0,1,Partida 8236 En esta Santa Iglecia de Santa Cr...,Santa Iglecia de Santa Cruz de Lorica,21,58,LOCATION
1,1,Partida 8236 En esta Santa Iglecia de Santa Cr...,veinte de Mayo de mil ochocien tos ochenta y c...,61,111,DATE
2,1,Partida 8236 En esta Santa Iglecia de Santa Cr...,José Maria Lugo,129,144,PERSON
3,1,Partida 8236 En esta Santa Iglecia de Santa Cr...,Eva Justina,182,193,PERSON
4,1,Partida 8236 En esta Santa Iglecia de Santa Cr...,cuatro de Febrero,207,224,DATE


In [ ]:
#no_test

collated_df = prodigy_output_to_collated_df("transcriptions/15834_annot.jsonl")
# Split data
train_df, valid_df, test_df = split_data_grp(collated_df, prop_train = 0.7, prop_validation = 0.2, grp_var = 'entry_no', seed=seed)
# Generate Spacy datasets
train_spacy = genSpaCyInput(train_df)
# Look at column names
train_df.head(10)

,entry_no,text,entity,start,end,label
13,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,Paula,10,15,PER
14,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,Esc.a,17,22,CHAR
15,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,Juebes veinte y tres de feb.o de mil sietec.to...,23,86,DATE
16,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,Thomas de Orvera,90,106,PER
17,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,Paula,145,150,PER
18,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,h. l.16,151,158,CHAR
19,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,Juan Joseph,162,173,PER
20,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,Maria Josepha,177,190,PER
21,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,esc.s,191,196,CHAR
22,2,[margin]: Paula. Esc.a Juebes veinte y tres de...,Capitan D. Luis Hurtado de Mendoza,201,235,PER


In [ ]:
#no_test

nlp = load_model()

for text, annots in train_spacy:
    if "[margin]: Fran.co esc.o Domingo dos de Marco" in text:
        print(spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), annots["entities"]))

['O', 'O', 'O', 'O', 'U-PER', 'U-CHAR', 'B-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'L-DATE', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'L-PER', 'U-CHAR', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'U-PER', 'U-CHAR', 'U-CHAR', 'O', 'O', 'O', 'U-CHAR', 'U-CHAR', 'O', 'B-PER', 'I-PER', 'I-PER', 'I-PER', 'L-PER', 'O', 'O', 'O', 'O', 'O', 'O', '-', '-', '-', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'L-PER', 'B-PER', 'I-PER', 'I-PER', 'I-PER', 'I-PER', 'L-PER']


C:\Users\genkindn\AppData\Local\Continuum\anaconda3\lib\site-packages\spacy\training\iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "[margin]: Fran.co esc.o Domingo dos de Marco de mi..." with entities "[(10, 17, 'PER'), (18, 23, 'CHAR'), (24, 78, 'DATE...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,


In [ ]:
#no_test
#Functioning code that this was all based off of

#spacy parameters
#cping = {'start':16, 'end':32, 'cp_rate':1.05}
#solves = {'learn_rate':0.001}
#dpout = 0.6
#save_mdir = 'models/exhaustive_models'

#load model from pretrained
#nlp_model = load_model()

#nlp_model, perf_df = model_meta_training(nlp_model, train_spacy, valid_df, verbose=True, save_dir = save_mdir,
#                                                 n_iter=10, solver_params=solves, compound_params=cping, dropout=dpout)
        
#display('Successful run')

# Plan
1. Run lots of trials varying learning rate and dropout, get a sense of what ranges might be the best
2. Repeat the above, but focused in on the identified areas
3. Repeat, but instead of varying hyper parameters, vary the training data

In [ ]:
#no_test

#if you want the same model everytime:
#random.seed(seed)
#fix_random_seed(seed)

#spacy parameters setup
#my_lr = [0.0001, 0.001, 0.01, 0.1]
#my_dps = [0.1, 0.3, 0.5, 0.7, 0.9]
my_lr = [.0001]
my_dps = [.1]

#spacy parameters
for i in range(len(my_lr)):
    cping = {'start':16, 'end':32, 'cp_rate':1.05}
    solves = {'learn_rate':my_lr[i]} #Original 0.001
    for j in range(len(my_dps)):
        dpout = my_dps[j] #Original 0.6
        save_mdir = 'models/exhaustive_models'

        #load model from pretrained
        nlp_model = load_model()

        nlp_model, perf_df = model_meta_training(nlp_model, train_spacy, valid_df, verbose=True, save_dir = save_mdir,
                                                 n_iter=10, solver_params=solves, compound_params=cping, dropout=dpout)
    
        perf_df['learning rate'] = my_lr[i]
        perf_df['dropout'] = my_dps[j]
        perf_df['compounding rate'] = cping['cp_rate']
        #Basis: schoolCopy = schoolDataFrame[allTheColumnsRequired].copy(deep=True)
        history_df = perf_df.copy(deep=True)
        
        #Going back and fixing this so that history_df is iterative
        
        
display('Successful run')

C:\Users\genkindn\AppData\Local\Continuum\anaconda3\lib\site-packages\spacy\training\iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "[margin]: Fran.co esc.o Domingo dos de Marco de mi..." with entities "[(10, 17, 'PER'), (18, 23, 'CHAR'), (24, 78, 'DATE...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,1.0,7140.336597,0.0,0.0,0.0


Saved model to models\exhaustive_models


C:\Users\genkindn\AppData\Local\Continuum\anaconda3\lib\site-packages\spacy\training\iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "[margin]: Fran.co esc.o Domingo dos de Marco de mi..." with entities "[(10, 17, 'PER'), (18, 23, 'CHAR'), (24, 78, 'DATE...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,2.0,2465.767262,14.893617,1.587302,2.868852


...

Performance hasn't improved for 4 cycles...


C:\Users\genkindn\AppData\Local\Continuum\anaconda3\lib\site-packages\spacy\training\iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "[margin]: Fran.co esc.o Domingo dos de Marco de mi..." with entities "[(10, 17, 'PER'), (18, 23, 'CHAR'), (24, 78, 'DATE...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,


,cycle_no,avg_cycle_loss,precision,recall,f_score
0,25.0,6.83896,88.127854,87.528345,87.827076


Performance hasn't improved for 5 cycles...
Done training after 25 meta cycles.


'Successful run'

In [ ]:
#no_test

#Manually working with the above data:
#I didn't configure history_df correctly originally so I manually upload this data:

lr_vec = [0.0001, 0.0001, 0.0001, 0.0001, 0.0001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.01, 0.01, 0.01, 0.01, 0.01, 0.1, 0.1, 0.1, 0.1, 0.1]
dpt_vec = [0.1, 0.3, 0.5, 0.7, 0.9, 0.1, 0.3, 0.5, 0.7, 0.9, 0.1, 0.3, 0.5, 0.7, 0.9, 0.1, 0.3, 0.5, 0.7, 0.9]
pre_vec = [89.183223, 89.910314, 90.227273, 0, 0, 89.686099, 89.532294, 89.662821, 88.487585, 0, 89.52164, 87.387387, 89.013453, 90.137615, 0, 88.564477, 84.009547, 53.82529, 0, 0]
rec_vec = [91.609977, 90.929705, 90.022676, 0, 0, 90.702948, 91.156463, 90.47619, 88.888889, 0, 89.115646, 87.981859, 90.022676, 89.115646, 0, 82.539683, 79.818594, 41.496599, 0, 0]
f_vec = [90.380313, 90.417136, 90.124858, 0, 0, 90.191657, 90.337079, 90.06772, 88.687783, 0, 89.318182, 87.683616, 89.51522, 89.623717, 0, 85.446009, 81.860465, 46.862996, 0, 0]

manual_data = {'Learning_Rate':lr_vec, 'Dropout':dpt_vec, 'Precision':pre_vec, 'Recall':rec_vec, 'F_Score':f_vec}

results_df = pd.DataFrame(manual_data)
results_df.head(20)

In [ ]:
suspect_df = results_df.loc[results_df['Recall'] == 0]
suspect_df.head(10)

Note that when dropout was 0.9, no results were generated.  Likewise, when either extreme of the learning rate (0.1 or 0.0001) were tested alongside the next highest dropout (0.7), there were also no results generated

In [ ]:
#no_test

#spacy parameters setup
my_lr = [0.0001, 0.001, 0.01, 0.1]
my_dps = [0.1, 0.3, 0.5, 0.7, 0.9]

#Note that these are dfs, need to be lists to use in plots
dps1 = results_df.loc[results_df['Dropout'] == 0.1]
dps3 = results_df.loc[results_df['Dropout'] == 0.3]
dps5 = results_df.loc[results_df['Dropout'] == 0.5]
dps7 = results_df.loc[results_df['Dropout'] == 0.7]
dps9 = results_df.loc[results_df['Dropout'] == 0.9]

lr0001 = results_df.loc[results_df['Learning_Rate'] == 0.0001]
lr001 = results_df.loc[results_df['Learning_Rate'] == 0.001]
lr01 = results_df.loc[results_df['Learning_Rate'] == 0.01]
lr1 = results_df.loc[results_df['Learning_Rate'] == 0.1]

print('dps3')
display(dps3.head())

print('lr0001')
display(lr0001.head())

In [ ]:
### DROP THE ROWS OF JUST 0s

print('Plot columns wrt same dropout, different learning rates')

plt.figure(1)
#plt.subplot(511)
dps1_precision = dps1['Precision'].tolist()
dps1_recall = dps1['Recall'].tolist()
dps1_f = dps1['F_Score'].tolist()
plt.plot(my_lr, dps1_precision, label='Precision')
plt.plot(my_lr, dps1_recall, label='Recall')
plt.plot(my_lr, dps1_f, label='F_Score')
plt.xlabel('Learning Rate')
plt.ylabel('Score')
plt.title('Dropout 0.1: Scores wrt Learning Rate')
plt.legend()
plt.show()

plt.figure(2)
#plt.subplot(511)
dps3_precision = dps3['Precision'].tolist()
dps3_recall = dps3['Recall'].tolist()
dps3_f = dps3['F_Score'].tolist()
plt.plot(my_lr, dps3_precision, label='Precision')
plt.plot(my_lr, dps3_recall, label='Recall')
plt.plot(my_lr, dps3_f, label='F_Score')
plt.xlabel('Learning Rate')
plt.ylabel('Score')
plt.title('Dropout 0.3: Scores wrt Learning Rate')
plt.legend()
plt.show()

plt.figure(3)
#plt.subplot(511)
dps5_precision = dps5['Precision'].tolist()
dps5_recall = dps5['Recall'].tolist()
dps5_f = dps5['F_Score'].tolist()
plt.plot(my_lr, dps5_precision, label='Precision')
plt.plot(my_lr, dps5_recall, label='Recall')
plt.plot(my_lr, dps5_f, label='F_Score')
plt.xlabel('Learning Rate')
plt.ylabel('Score')
plt.title('Dropout 0.5: Scores wrt Learning Rate')
plt.legend()
plt.show()

plt.figure(4)
#plt.subplot(511)
dps7_precision = dps7['Precision'].tolist()
dps7_recall = dps7['Recall'].tolist()
dps7_f = dps7['F_Score'].tolist()
plt.plot(my_lr, dps7_precision, label='Precision')
plt.plot(my_lr, dps7_recall, label='Recall')
plt.plot(my_lr, dps7_f, label='F_Score')
plt.xlabel('Learning Rate')
plt.ylabel('Score')
plt.title('Dropout 0.7: Scores wrt Learning Rate')
plt.legend()
plt.show()


Repeating the above with respect to learning rate.  Note that we expect the lower learning rates to have the best scores

In [ ]:
#no_test

my_lr = [0.0001, 0.001, 0.01, 0.1]
my_dps = [0.1, 0.3, 0.5]

#Note: added [0:3] to the backs of tolist() in order to drop the dropouts of 0
print('Plot columns wrt same learning rates, different dropout')
plt.figure(1)
#plt.subplot(511)
lr0001_precision = lr0001['Precision'].tolist()[0:3]
lr0001_recall = lr0001['Recall'].tolist()[0:3]
lr0001_f = lr0001['F_Score'].tolist()[0:3]
plt.plot(my_dps, lr0001_precision, label='Precision')
plt.plot(my_dps, lr0001_recall, label='Recall')
plt.plot(my_dps, lr0001_f, label='F_Score')
plt.xlabel('Dropout')
plt.ylabel('Score')
plt.title('Learning Rate 0.0001: Scores wrt Dropout')
plt.legend()
plt.show()

plt.figure(2)
#plt.subplot(511)
lr001_precision = lr001['Precision'].tolist()[0:3]
lr001_recall = lr001['Recall'].tolist()[0:3]
lr001_f = lr001['F_Score'].tolist()[0:3]
plt.plot(my_dps, lr001_precision, label='Precision')
plt.plot(my_dps, lr001_recall, label='Recall')
plt.plot(my_dps, lr001_f, label='F_Score')
plt.xlabel('Dropout')
plt.ylabel('Score')
plt.title('Learning Rate 0.0001: Scores wrt Dropout')
plt.legend()
plt.show()

Repeat the above but focus in on the plateua around 90, ignore the drop to 0

In [ ]:
#no_test

#Updated to drop ones we don't care about
my_lr = [0.0001, 0.001, 0.01, 0.1]
my_dps = [0.1, 0.3, 0.5]

print('Plot columns wrt same learning rates, different dropout')
plt.figure(1)
#plt.subplot(511)
lr0001_precision = lr0001['Precision'].tolist()[0:3]
lr0001_recall = lr0001['Recall'].tolist()[0:3]
lr0001_f = lr0001['F_Score'].tolist()[0:3]
plt.plot(my_dps, lr0001_precision, label='Precision')
plt.plot(my_dps, lr0001_recall, label='Recall')
plt.plot(my_dps, lr0001_f, label='F_Score')
plt.xlabel('Dropout')
plt.ylabel('Score')
plt.title('Learning Rate 0.0001: Scores wrt Dropout')
plt.show()

plt.figure(2)
#plt.subplot(511)
lr001_precision = lr001['Precision'].tolist()[0:3]
lr001_recall = lr001['Recall'].tolist()[0:3]
lr001_f = lr001['F_Score'].tolist()[0:3]
plt.plot(my_dps, lr001_precision, label='Precision')
plt.plot(my_dps, lr001_recall, label='Recall')
plt.plot(my_dps, lr001_f, label='F_Score')
plt.xlabel('Dropout')
plt.ylabel('Score')
plt.title('Learning Rate 0.0001: Scores wrt Dropout')
plt.show()

In [ ]:
#no_test

#This block hasn't been written yet, will do the above but as a callable function
#Note that history_df is wrong, needs to be updated on every run, not just overwriting a new copy...

#load saved model and performance metrics if desired:
#this_model_dir = 'models/42-initial-model/4e0f496_dp0.4_lr0.0003'
#nlp_model = load_model(this_model_dir)
#perf_df = read_csv(this_model_dir + '\perf_df.csv')

#perf_df.set_index('cycle_no', drop=True, inplace=True)
#perf_df.head()

#Plotting
#Columns: cycle_no avg_cycle_loss precission recall f_score dropout learning_rate

#Note that training curves already exist in NB 42:

#scale the losses a bit just to see things together
#perf_df['loss_scaled'] = perf_df['avg_cycle_loss']/10
#perf_df.drop(columns=['avg_cycle_loss']).plot()

In [ ]:
#no_test

from nbdev.export import notebook2script
notebook2script()